<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-films/blob/Dodawanie_kolumny_z_epokami/colabs/Dodawanie_kolumny_z_epokami.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wczytanie danych

In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark2.4.5
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
# install findspark
!pip install -q findspark
# clone github repo
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-films
# Przeniesienie plików z BigData2022-films do katalogu nadrzędnego
!mv BigData2022-films/* .
!mv BigData2022-films/.* .
!rmdir BigData2022-films
     
import os

# setup environment variables for our Spark Session to work
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.2.1-bin-hadoop3.2'

from lib.pyspark_startup import init, load

spark = init()
     
!chmod +x data_source.sh
!./data_source.sh

df = load(spark)
!git checkout -f Dodawanie_kolumny_z_epokami

Cloning into 'BigData2022-films'...
remote: Enumerating objects: 593, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 593 (delta 132), reused 79 (delta 71), pack-reused 385
Receiving objects: 100% (593/593), 2.86 MiB | 9.25 MiB/s, done.
Resolving deltas: 100% (308/308), done.
mv: cannot move 'BigData2022-films/.' to './.': Device or resource busy
mv: cannot move 'BigData2022-films/..' to './..': Device or resource busy
--2022-12-16 18:26:52--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.35.24.124, 13.35.24.56, 13.35.24.122, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.35.24.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237981379 (227M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 226.96M  21.9MB/s    in 12s     

2022-12-16 18:27:05 (19.3 MB/s) - ‘nam

In [ ]:
df.show(10)

+---------+----------------+--------------------+-----------------+---------------------+-------------------------+-----------------------+--------------------+--------------+--------------------------+-----+-------------+---------+----+---------+---------+---------+---------+----+----+----+----+
|       id|rodzaj_produkcji|               tytul|czy_dla_doroslych|rok_wydania_produkcji|rok_zakonczenia_produkcji|dlugosc_produkcji_w_min|             gatunek|liczba_sezonow|liczba_wszystkich_odcinkow|ocena|liczba_glosow|        1|  10|        2|        3|        4|        5|   6|   7|   8|   9|
+---------+----------------+--------------------+-----------------+---------------------+-------------------------+-----------------------+--------------------+--------------+--------------------------+-----+-------------+---------+----+---------+---------+---------+---------+----+----+----+----+
|tt0000002|           short|Le clown et ses c...|                0|                 1892|                 

# Funkcja dodająca kolumnę z epokami

In [5]:
from pyspark.sql.functions import col

def df_column_string_to_int(df: DataFrame, column_name: str) -> DataFrame:
  df_int_column = df.withColumn(column_name,col(column_name).cast("int"))
  return df_int_column

In [13]:
from pyspark.sql.functions import col, when
from pyspark.sql import DataFrame
from functools import reduce


def add_epoch_column(df: DataFrame, periods = [1901,1918,1926,1939,1954,1970,1985,1994,2009]) -> DataFrame:

  assert df.filter(df.rok_wydania_produkcji == "\\N").count() == 0
  assert dict(df.dtypes)["rok_wydania_produkcji"] == "int"
 
  formulas = [ 
    when(col('rok_wydania_produkcji') <= period, str(index+1))  
    for period, index 
    in zip(periods, range(len(periods)))]


  df_periods = df.withColumn('epoka',
    (reduce(lambda x,y: x.y, formulas)).otherwise( len(periods) +1)
  )  

  return df_periods

In [12]:
df.show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [14]:
sample = df_column_string_to_int(df.filter(df.id == 'tt0111161'),"rok_wydania_produkcji")
sample_2 = df.filter(df.id == "tt0080155")

add_epoch_column(sample)
add_epoch_column(sample_2)

IllegalArgumentException: ignored

## Test

In [ ]:
import findspark

findspark.init()

import pytest
import pyspark

from pyspark.sql import SparkSession, DataFrame
from pyspark_test import assert_pyspark_df_equal
from lib.feature_creators import add_epoch_column

def test_add_epoch_column():

  spark = SparkSession.builder.master("local[*]").getOrCreate()
  df = spark.createDataFrame(
      [
          (1, 1900),
          (2, 1917),
          (3, 1920),
          (4, 1938),
          (5, 1953),
          (6, 1960),
          (7, 1984),
          (8, 1993),
          (9, 2008),
          (10, 2022)
      ],  
      "id int, rok_wydania_produkcji int",
  )

  expect_df = spark.createDataFrame(
      [
          (1, 1900, "1"),
          (2, 1917, "2"),
          (3, 1920, "3"),
          (4, 1938, "4"),
          (5, 1953, "5"),
          (6, 1960, "6"),
          (7, 1984, "7"),
          (8, 1993, "8"),
          (9, 2008, "9"),
          (10, 2022, "10")
      ],  
      "id int, rok_wydania_produkcji int, epoka string",
  )

  result = add_epoch_column(df)
  assert_pyspark_df_equal(result, expect_df)

AnalysisException: ignored